# Data Cleaning and Preprocessing

To use filtered data and produce cleaned data, do the following:  
1. From filtered_data, bring your desired data set
2. Run the code cells for your desired cleaning operation
3. Save your cleaned data and place in cleaned_data

## Cleaning aircraft data

In [1]:
import pandas as pd

# Load data
filtered_aircraft = pd.read_csv('filtered_data/filtered_aircraft_20250301.csv')

filtered_aircraft.head()

,timestamp,hex,type,flight,r,t,alt_baro,gs,track,baro_rate,squawk,category,lat,lon,nic,rc,seen_pos,messages,seen,rssi
0,2025-03-01 00:00:00,ac0818,adsb_icao,SWA3036,N8744B,B38M,ground,0.0,NaN,NaN,2762.0,A3,21.330677,-157.920564,8.0,186.0,0.081,75850756,0.1,-19.6
1,2025-03-01 00:00:00,a7888c,adsb_icao,ASA9662,N585AS,B738,1425,147.0,89.22,-1088.0,5210.0,A3,61.169495,-150.086485,8.0,186.0,0.014,71684417,0.0,-11.4
2,2025-03-01 00:00:00,a239e0,adsb_icao,NaN,N24202,B738,ground,0.0,NaN,NaN,NaN,A3,37.611809,-122.381925,8.0,186.0,12.938,59852756,12.9,-2.3
3,2025-03-01 00:00:00,a446aa,adsb_icao,DAL2613,N375DA,B738,775,169.0,0.68,2496.0,3542.0,A3,47.461441,-122.307701,8.0,186.0,0.284,82460165,0.1,-11.7
4,2025-03-01 00:00:00,a03b6d,adsb_icao,DAL673,N114DU,BCS1,ground,0.1,NaN,NaN,621.0,A3,47.441630,-122.304470,8.0,186.0,2.850,80004333,2.9,-4.6


In [2]:
# Drop unnecessary columns
filtered_aircraft.drop('type', axis=1, inplace=True)        # Drop the 'type' column
filtered_aircraft.drop('hex', axis=1, inplace=True)         # Drop the 'hex' column
filtered_aircraft.drop('category', axis=1, inplace=True)    # Drop the 'category' column
filtered_aircraft.drop('nic', axis=1, inplace=True)         # Drop the 'nic' column
filtered_aircraft.drop('r', axis=1, inplace=True)           # Drop the 'r' column
filtered_aircraft.drop('squawk', axis=1, inplace=True)      # Drop the 'squawk' column
filtered_aircraft.drop('seen_pos', axis=1, inplace=True)    # Drop the 'seen_pos' column
filtered_aircraft.drop('messages', axis=1, inplace=True)    # Drop the 'messages' column
filtered_aircraft.drop('seen', axis=1, inplace=True)        # Drop the 'seen' column
filtered_aircraft.drop('rc', axis=1, inplace=True)          # Drop the 'rc' column
filtered_aircraft.drop('rssi', axis=1, inplace=True)        # Drop the 'rssi' column

filtered_aircraft.head()

,timestamp,flight,t,alt_baro,gs,track,baro_rate,lat,lon
0,2025-03-01 00:00:00,SWA3036,B38M,ground,0.0,NaN,NaN,21.330677,-157.920564
1,2025-03-01 00:00:00,ASA9662,B738,1425,147.0,89.22,-1088.0,61.169495,-150.086485
2,2025-03-01 00:00:00,NaN,B738,ground,0.0,NaN,NaN,37.611809,-122.381925
3,2025-03-01 00:00:00,DAL2613,B738,775,169.0,0.68,2496.0,47.461441,-122.307701
4,2025-03-01 00:00:00,DAL673,BCS1,ground,0.1,NaN,NaN,47.441630,-122.304470


In [3]:
airport_elevation = 17  # Airport elevation in feet

# Convert timestamp to datetime format
filtered_aircraft['timestamp'] = pd.to_datetime(filtered_aircraft['timestamp'])

# Convert "ground" in barometric altitude to airport elevation (17 ft)
filtered_aircraft['alt_baro'] = filtered_aircraft['alt_baro'].replace('ground', airport_elevation)

# Convert barometric altitude to integer, keeping NaN as NaN
filtered_aircraft['alt_baro'] = pd.to_numeric(filtered_aircraft['alt_baro'], errors='coerce').astype('Int64')

# Convert barometric altitude to altitude above ground level
filtered_aircraft['alt'] = filtered_aircraft['alt_baro'] - airport_elevation

# Round values within +30 feet and set any negative altitudes to 0
filtered_aircraft['alt'] = filtered_aircraft['alt'].apply(lambda x: 0 if pd.isna(x) or x < 0 or x <= 30 else x)

filtered_aircraft['alt'] = pd.to_numeric(filtered_aircraft['alt'], errors='coerce').astype('Int64')

filtered_aircraft.drop('alt_baro', axis=1, inplace=True)  # Drop the 'alt_baro' column

# Convert ground speed to float, keeping NaN as NaN
filtered_aircraft['gs'] = pd.to_numeric(filtered_aircraft['gs'], errors='coerce')

# Convert vertical rate to float, keeping NaN as NaN
filtered_aircraft['baro_rate'] = pd.to_numeric(filtered_aircraft['baro_rate'], errors='coerce')

# Convert latitude and longitude to float, keeping NaN as NaN
filtered_aircraft['lat'] = pd.to_numeric(filtered_aircraft['lat'], errors='coerce')
filtered_aircraft['lon'] = pd.to_numeric(filtered_aircraft['lon'], errors='coerce')

filtered_aircraft.head()

,timestamp,flight,t,gs,track,baro_rate,lat,lon,alt
0,2025-03-01,SWA3036,B38M,0.0,NaN,NaN,21.330677,-157.920564,0
1,2025-03-01,ASA9662,B738,147.0,89.22,-1088.0,61.169495,-150.086485,1408
2,2025-03-01,NaN,B738,0.0,NaN,NaN,37.611809,-122.381925,0
3,2025-03-01,DAL2613,B738,169.0,0.68,2496.0,47.461441,-122.307701,758
4,2025-03-01,DAL673,BCS1,0.1,NaN,NaN,47.441630,-122.304470,0


In [4]:
# Filter by position; only include aircraft within roughly 10nm of the airport

north_limit = 32.8996
south_limit = 32.5662
east_limit = -116.9916
west_limit = -117.3878

filtered_aircraft = filtered_aircraft[
    (filtered_aircraft['lat'] <= north_limit) &
    (filtered_aircraft['lat'] >= south_limit) &
    (filtered_aircraft['lon'] <= east_limit) &
    (filtered_aircraft['lon'] >= west_limit)
]

filtered_aircraft.reset_index(drop=True, inplace=True)

filtered_aircraft.head()

,timestamp,flight,t,gs,track,baro_rate,lat,lon,alt
0,2025-03-01,ACA1013,A321,0.0,314.0,0.0,32.733467,-117.206653,0
1,2025-03-01,UAL2117,B738,0.0,NaN,NaN,32.735184,-117.203288,0
2,2025-03-01,SKW3473,E75L,0.0,NaN,NaN,32.733043,-117.200691,0
3,2025-03-01,SKW3320,E75L,0.0,NaN,NaN,32.732252,-117.200060,0
4,2025-03-01,SWA4363,B737,0.0,208.0,0.0,32.733765,-117.198169,0


In [5]:
# Filter aircraft by altitude; only include aircraft below 10,000 feet
filtered_aircraft = filtered_aircraft[filtered_aircraft['alt'] < 6000]

filtered_aircraft.reset_index(drop=True, inplace=True)

filtered_aircraft.head()

,timestamp,flight,t,gs,track,baro_rate,lat,lon,alt
0,2025-03-01,ACA1013,A321,0.0,314.0,0.0,32.733467,-117.206653,0
1,2025-03-01,UAL2117,B738,0.0,NaN,NaN,32.735184,-117.203288,0
2,2025-03-01,SKW3473,E75L,0.0,NaN,NaN,32.733043,-117.200691,0
3,2025-03-01,SKW3320,E75L,0.0,NaN,NaN,32.732252,-117.200060,0
4,2025-03-01,SWA4363,B737,0.0,208.0,0.0,32.733765,-117.198169,0


In [6]:
# Sort by timestamp
filtered_aircraft.sort_values(by='timestamp', inplace=True)

filtered_aircraft.reset_index(drop=True, inplace=True)

filtered_aircraft.head()

,timestamp,flight,t,gs,track,baro_rate,lat,lon,alt
0,2025-03-01,ACA1013,A321,0.0,314.00,0.0,32.733467,-117.206653,0
1,2025-03-01,SWA4157,B38M,144.0,286.55,-832.0,32.729324,-117.172101,158
2,2025-03-01,SKW3378,E75L,0.0,NaN,NaN,32.729517,-117.176939,0
3,2025-03-01,EJA838,C700,0.0,14.00,0.0,32.733450,-117.178396,0
4,2025-03-01,SWA3095,B737,0.0,NaN,NaN,32.733112,-117.195735,0


In [7]:
# Save cleaned data

filtered_aircraft.to_csv('cleaned_data/aircraft_cleaned_20250301.csv', index=False)

## Cleaning operations data

In [8]:
# Load operations data
filtered_operations = pd.read_csv('filtered_data/filtered_operations_20250301.csv')

filtered_operations.head()

,time,icao,operation,airport,registration,flight,ac_type,runway,flight_link,squawk,...,mil,apt_type,name,continent,iso_country,iso_region,municipality,scheduled_service,iata_code,elev
0,2025-03-01 00:00:38,ac3315,landing,KSAN,N8852Q,SWA4157,B38M,27.0,https://globe.adsbexchange.com/?icao=ac3315&zo...,2762.0,...,f,large_airport,San Diego International Airport,NaN,US,US-CA,NaN,yes,SAN,17
1,2025-03-01 00:02:05,a524c5,takeoff,KSAN,N430SY,SKW3378,E75L,27.0,https://globe.adsbexchange.com/?icao=a524c5&zo...,7266.0,...,f,large_airport,San Diego International Airport,NaN,US,US-CA,NaN,yes,SAN,17
2,2025-03-01 00:04:27,407fcc,landing,KSAN,G-XWBR,BAW82P,A35K,27.0,https://globe.adsbexchange.com/?icao=407fcc&zo...,1420.0,...,f,large_airport,San Diego International Airport,NaN,US,US-CA,NaN,yes,SAN,17
3,2025-03-01 00:08:37,acda2c,landing,KSAN,N927WN,SWA3338,B737,27.0,https://globe.adsbexchange.com/?icao=acda2c&zo...,3302.0,...,f,large_airport,San Diego International Airport,NaN,US,US-CA,NaN,yes,SAN,17
4,2025-03-01 00:09:33,ab7781,takeoff,KSAN,N838QS,EJA838,C700,27.0,https://globe.adsbexchange.com/?icao=ab7781&zo...,2427.0,...,f,large_airport,San Diego International Airport,NaN,US,US-CA,NaN,yes,SAN,17


In [9]:
# Remove unnecessary columns from operations data
filtered_operations.drop(['icao',
                        'airport',
                        'registration',
                        'flight_link',
                        'squawk',
                        'signal_type',
                        'category',
                        'year',
                        'manufacturer',
                        'model',
                        'ownop',
                        'faa_pia',
                        'faa_ladd',
                        'short_type',
                        'mil',
                        'apt_type',
                        'name',
                        'continent',
                        'iso_country',
                        'iso_region',
                        'municipality',
                        'scheduled_service',
                        'iata_code',
                        'elev'], axis=1, inplace=True)

filtered_operations.head()

,time,operation,flight,ac_type,runway
0,2025-03-01 00:00:38,landing,SWA4157,B38M,27.0
1,2025-03-01 00:02:05,takeoff,SKW3378,E75L,27.0
2,2025-03-01 00:04:27,landing,BAW82P,A35K,27.0
3,2025-03-01 00:08:37,landing,SWA3338,B737,27.0
4,2025-03-01 00:09:33,takeoff,EJA838,C700,27.0


In [10]:
# Convert time to datetime format
filtered_operations['time'] = pd.to_datetime(filtered_operations['time'], format='%Y-%m-%d %H:%M:%S', errors='coerce')

# Convert runway to integer, keeping NaN as NaN
filtered_operations['runway'] = pd.to_numeric(filtered_operations['runway'], errors='coerce').astype('Int64')

filtered_operations.head()

,time,operation,flight,ac_type,runway
0,2025-03-01 00:00:38,landing,SWA4157,B38M,27
1,2025-03-01 00:02:05,takeoff,SKW3378,E75L,27
2,2025-03-01 00:04:27,landing,BAW82P,A35K,27
3,2025-03-01 00:08:37,landing,SWA3338,B737,27
4,2025-03-01 00:09:33,takeoff,EJA838,C700,27


In [11]:
# Save data
filtered_operations.to_csv('cleaned_data/operations_cleaned_20250301.csv', index=False)